In [1]:
import pandas as pd

## Function definitions

In [2]:
def group_by(data, group_column = 'user_id', target_column = 'user_journey', sessions = 'all', count_from = 'last'):
    """
    Groups the data in a DataFrame by the group_column and concatenates all data in target_column for each group.
    All other columns are set to their values in the first record of the group.
    Can also choose whether to aggregate all records for each group or only the first or last sessions.
    Similar to GROUP BY group_column + GROUP_CONCAT(target_column) in SQL.
    
        Parameters:
            data (pandas.DataFrame): The DataFrame containing the data to be grouped
            
            group_column (str): The columnn from which to create the groups
                Default: 'user_id', the expected column to group by
            
            target_column (str): The column which is to be concatenated
                Default: 'user_journey', the expected name of the column
                
            sessions (str or int): If int, indicates the number of records to aggregate (concat) in the target_column.
                Depending on count_from, it starts aggregating the records from the first or last element.
                Can also be a string with value 'all' or 'all_except_last'.
                Default: 'all', includes all data in the aggregation
                
            count_from (str): Possible values - 'last' and 'first'. Indicates from where to count when sessions is int
                Default: 'last'
            
        
        Returns:
            df (pandas.DataFrame): A new DataFrame containing the grouped data. The original DataFrame is not modified.
    """
    
    
    # Check for correct arguments' type
    if not isinstance(data, pd.DataFrame):
        raise TypeError("'data' should be of type 'pandas.DataFrame': received {} instead".format(type(data)))
    
    if not isinstance(group_column, str):
        raise TypeError("'group_column' should be of type 'str': received {} instead".format(type(group_column)))
    
    if not isinstance(target_column, str):
        raise TypeError("'target_column' should be of type 'str': received {} instead".format(type(target_column)))
    
    if not (isinstance(sessions, str) or isinstance(sessions, int)):
        raise TypeError("'sessions' should be of type 'str' or 'int': received {} instead".format(type(sessions)))
    
    if not isinstance(count_from, str):
        raise TypeError("'count_from' should be of type 'str': received {} instead".format(type(count_from)))
    
    
    # Check for correct arguments' value
    if group_column not in data.columns:
        raise ValueError("No column named '{}' in the data".format(group_column))
    
    if target_column not in data.columns:
        raise ValueError("No column named '{}' in the data".format(target_column))
    
    if isinstance(sessions, str) and sessions != "all" and sessions != "all_except_last":
        raise ValueError("Possible 'session' string values are: 'all' and 'all_except_last'")
    
    if isinstance(sessions, int) and sessions < 0:
        raise ValueError("Sessions must be a non-negative integer")
    
    if count_from != "last" and count_from != "first":
        raise ValueError("Possible 'count_from' values are: 'last' and 'first'")
    
    
    
    ### MAIN FUNCTION BODY ###
    

    
    # Create a new dataframe to populate with the results
    df = pd.DataFrame(columns = data.columns)
    
    
    # Set the start and end index for the sessions that should be aggregated
    if sessions == "all":
        start = 0
        end = None
    
    elif sessions == "all_except_last":
        start = 0
        end = -1
    
    elif count_from == "last":
        start = - sessions
        end = None
    
    elif count_from == "first":
        start = 0
        end = sessions
    
    
    # Obtain the set of unique user_id values
    groups = set(data[group_column])
    
    # Iterate over each group (unordered, as sets are unordered)
    for group_value in groups:
        
        group_mask = list(data[group_column] == group_value) # create a mask to filter only the rows with group_value
        group_table = data[group_mask] # a view of the data corresponding to a single group
        
        # Obtain an aggregated user journey string corresponding to the group
        user_journey = "-".join(list(group_table[target_column])[start:end])
        
        # Append a row to the new DataFrame corresponding to a single (aggregated) group
        new_index = len(df)
        df.loc[new_index] = group_table.iloc[0].copy() # all values except target_column are copied from the first occurence
        df.loc[new_index][target_column] = user_journey

    
    # Reset the index (precautionary) and sort the new DataFrame by user_id
    df.sort_values(by=[group_column], ignore_index = True, inplace = True)
    df.reset_index(drop = True, inplace = True)
    
    
    
    return df
    

In [3]:
def remove_pages(data, pages = [], target_column = 'user_journey'):
    """
    Returns a new DataFrame where the provided list of pages is to be removed from the user journey strings.
    
        Parameters:
            data (pandas.DataFrame): The DataFrame containing the user journeys data
            
            pages (list or set): A list or set of strings, representing the pages to be removed from the user journeys
                Default: [], an empty list, no pages to remove
            
            target_column (str): The column containing the user journey strings
                Default: 'user_journey', the expected name of the column
            
        
        Returns:
            df (pandas.DataFrame): A new DataFrame containing the data. The original DataFrame is not modified.
    """
    
    
    # Check for correct arguments' type
    if not isinstance(data, pd.DataFrame):
        raise TypeError("'data' should be of type 'pandas.DataFrame': received {} instead".format(type(data)))
    
    if not (isinstance(pages, list) or isinstance(pages, set)):
        raise TypeError("'pages' should be of type 'list' or 'set': received {} instead".format(type(pages)))
    
    for item in pages:
        if not isinstance(item, str):
            raise TypeError("'pages' should be a list of 'str': received list of {} instead".format(type(item)))
    
    if not isinstance(target_column, str):
        raise TypeError("'target_column' should be of type 'str': received {} instead".format(type(target_column)))
    
    
    # Check for correct arguments' value
    if target_column not in data.columns:
        raise ValueError("No column named '{}' in the data".format(target_column))
    
    
    
    ### MAIN FUNCTION BODY ###
    
    
    
    # Make a copy of the dataframe as to not accidentaly modify the original data
    df = data.copy()
    pages = set(pages)
    
    if len(pages) == 0:
        return df
    
    # Obtain a list of all user journey strings -> ["page1-page2-...pageN", ...]
    user_journey = list(df[target_column])
    
    # Split the journey strings into pages -> [["page1", "page2", ..."pageN"], ...]
    user_journey = [journey.split('-') for journey in user_journey]
    
    # Include only pages that are not contained in the set of pages to remove
    user_journey = [[page for page in journey if page not in pages] for journey in user_journey]
    
    # Combine the pages into strings again -> ["page1-page2-...pageN", ...]
    user_journey = ["-".join(i) for i in user_journey]
    
    
    # Update the user journey column with the new strings
    df[target_column] = user_journey
    
    
    
    return df
    

In [4]:
def remove_page_duplicates(data, target_column = 'user_journey'):
    """
    Returns a new DataFrame where consecutive page duplicates are removed from the user journey strings.
    Note that if the page duplicates are not consecutive, they are not removed.
    
        Parameters:
            data (pandas.DataFrame): The DataFrame containing the user journeys data
            
            target_column (str): The column containing the user journey strings
                Default: 'user_journey', the expected name of the column
            
        
        Returns:
            df (pandas.DataFrame): A new DataFrame containing the data. The original DataFrame is not modified.
    """
    
    
    # Check for correct arguments' type
    if not isinstance(data, pd.DataFrame):
        raise TypeError("'data' should be of type 'pandas.DataFrame': received {} instead".format(type(data)))
    
    if not isinstance(target_column, str):
        raise TypeError("'target_column' should be of type 'str': received {} instead".format(type(target_column)))
    
    
    # Check for correct arguments' value
    if target_column not in data.columns:
        raise ValueError("No column named '{}' in the data".format(target_column))
    
    
    
    ### MAIN FUNCTION BODY ###
    
    
    
    # Make a copy of the dataframe as to not accidentaly modify the original data
    df = data.copy()
    
    
    # Obtain a list of all user journey strings -> ["page1-page2-...pageN", ...]
    user_journey = list(df[target_column])
    
    # Split the journey strings into pages -> [["page1", "page2", ..."pageN"], ...]
    user_journey = [journey.split('-') for journey in user_journey]
    
    
    # Loop through all pages and remove consecutive duplicates
    for i in range(len(user_journey)):
        j = 1
        
        while(j < len(user_journey[i])):
            
            if user_journey[i][j-1] == user_journey[i][j]: # Check and remove the page if the next one is the same
                user_journey[i].pop(j)
            
            else:
                j += 1 # Move to the next page
    
    
    # Combine the pages into strings again -> ["page1-page2-...pageN", ...]
    user_journey = ["-".join(i) for i in user_journey]
    
    
    # Update the user journey column with the new strings
    df[target_column] = user_journey
    
    
    
    return df
    

## Main

In [5]:
# Load the unprocessed user journey data (SQL query result)
raw_data = pd.read_csv('user_journey_raw.csv')

In [6]:
# Check how the data looks
raw_data.head()

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,2980248,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,2992252,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,3070491,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,3709807,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...


In [7]:
# Create a copy of the raw data that will be updated and drop the "session_id" column as we don't need it
clean_data = raw_data.copy().drop("session_id", axis=1)

In [8]:
# Preprocessing of the data
clean_data = group_by(clean_data)
clean_data = remove_pages(clean_data, [])
clean_data = remove_page_duplicates(clean_data)

In [9]:
clean_data.head()

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Other-Sign up-Log in-Homepage-...
1,3395,Annual,Other-Pricing-Sign up-Log in-Homepage-Pricing-...
2,10107,Annual,Homepage-Career tracks-Homepage-Career tracks-...
3,11145,Monthly,Homepage-Log in-Homepage-Log in-Homepage-Log i...
4,12400,Monthly,Homepage-Career tracks-Sign up-Log in-Other-Ca...


In [10]:
# Save the preprocessed data
clean_data.to_csv('user_journey.csv', index = False)